<a href="https://www.kaggle.com/code/sitbayevalibek/new-york-home-predict?scriptVersionId=132077889" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housing-prices-dataset/Housing.csv


In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv("/kaggle/input/housing-prices-dataset/Housing.csv")
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [5]:
df['mainroad'] = df['mainroad'].map({'yes': 1, 'no': 0})
df['guestroom'] = df['guestroom'].map({'yes': 1, 'no': 0})
df['basement'] = df['basement'].map({'yes': 1, 'no': 0})
df['hotwaterheating'] = df['hotwaterheating'].map({'yes': 1, 'no': 0})
df['airconditioning'] = df['airconditioning'].map({'yes': 1, 'no': 0})
df['prefarea'] = df['prefarea'].map({'yes': 1, 'no': 0})

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    int64 
 6   guestroom         545 non-null    int64 
 7   basement          545 non-null    int64 
 8   hotwaterheating   545 non-null    int64 
 9   airconditioning   545 non-null    int64 
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    int64 
 12  furnishingstatus  545 non-null    object
dtypes: int64(12), object(1)
memory usage: 55.5+ KB


In [7]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
h = train_set.drop('price', axis=1)
h_y = train_set['price'].copy()

In [9]:
h_num = h.drop('furnishingstatus', axis=1)
h_cat = h[['furnishingstatus']]

In [10]:
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

In [11]:
num_pipeline.fit_transform(h_num)

array([[ 0.38416819,  0.05527092,  1.53917323, ...,  1.50124327,
         0.36795665, -0.55262032],
       [ 0.9291807 ,  0.05527092,  1.53917323, ...,  1.50124327,
         2.70998729, -0.55262032],
       [-0.60775457, -1.28351359, -0.5579503 , ...,  1.50124327,
         1.53897197, -0.55262032],
       ...,
       [-0.29709744,  0.05527092,  1.53917323, ..., -0.66611456,
         0.36795665, -0.55262032],
       [-0.5060189 , -1.28351359, -0.5579503 , ..., -0.66611456,
        -0.80305867, -0.55262032],
       [ 0.15707965,  0.05527092,  1.53917323, ...,  1.50124327,
         0.36795665, -0.55262032]])

In [12]:
num_attr = list(h_num)
cat_attr = ['furnishingstatus']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attr),
    ('cat', OneHotEncoder(), cat_attr)
])

In [13]:
h_prepared = full_pipeline.fit_transform(h)

In [14]:
LR_model = LinearRegression()
LR_model.fit(h_prepared, h_y)

LinearRegression()

In [15]:
test_data = h.sample(20)
test_label = h_y.loc[test_data.index]

In [16]:
#tdp = test_data_prepared (abbreviation)
tdp = full_pipeline.transform(test_data)

In [17]:
predicted_data = LR_model.predict(tdp)

In [18]:
r = pd.DataFrame({'Predicted LR_model':(predicted_data).round(2), 'Real Price': test_label})
r['Predicted LR_model'] = r['Predicted LR_model'].map('{:.0f}'.format)
pd.DataFrame(r)

,Predicted LR_model,Real Price
294,4387983,4200000
87,4007699,6475000
511,2329026,2520000
35,7569977,8080940
116,5920946,6020000
141,6860174,5652500
376,5159484,3640000
539,2432255,1855000
309,4602018,4130000
328,5610867,3990000


In [19]:
h_test = test_set.drop('price', axis=1)
h_y_test = test_set['price'].copy()

In [20]:
#h_test_pre = h test prepared (abbreviation)
h_test_pre = full_pipeline.transform(h_test)

In [21]:
h_y_predicted = LR_model.predict(h_test_pre)

In [22]:
lin_mse = mean_squared_error(h_y_test, h_y_predicted)
# RMSE 
lin_rmse = np.sqrt(lin_mse)
print(f"{lin_rmse:.0f}")

1324507


In [23]:
RF_model = RandomForestRegressor()
RF_model.fit(h_prepared, h_y)

RandomForestRegressor()

In [24]:
RFtest_data = h.sample(20)
RFtest_label = h_y.loc[RFtest_data.index]

In [25]:
#RFtdp = test_data_prepared (abbreviation)
RFtdp = full_pipeline.transform(RFtest_data)

In [26]:
RFpredicted_data = RF_model.predict(RFtdp)

In [27]:
RFr = pd.DataFrame({'Predicted RF_model':(predicted_data).round(2), 'Real Price': test_label})
RFr['Predicted RF_model'] = RFr['Predicted RF_model'].map('{:.0f}'.format)
pd.DataFrame(RFr)

,Predicted RF_model,Real Price
294,4387983,4200000
87,4007699,6475000
511,2329026,2520000
35,7569977,8080940
116,5920946,6020000
141,6860174,5652500
376,5159484,3640000
539,2432255,1855000
309,4602018,4130000
328,5610867,3990000


In [28]:
RFh_test = test_set.drop('price', axis=1)
RFh_y_test = test_set['price'].copy()

In [29]:
#RFh_test_pre = h test prepared (abbreviation)
RFh_test_pre = full_pipeline.transform(RFh_test)

In [30]:
RFh_y_predicted = RF_model.predict(RFh_test_pre)

In [31]:
RFlin_mse = mean_squared_error(RFh_y_test, RFh_y_predicted)
# RMSE 
RFlin_rmse = np.sqrt(RFlin_mse)
print(f"{RFlin_rmse:.0f}")

1386958
